In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker 
import matplotlib.colors as mcolors
from matplotlib import colormaps
from matplotlib.colors import Normalize
from scipy import fftpack
from scipy.sparse import csr_matrix
import scipy.sparse
# Importing fft functions from scipy
from scipy.fft import fft, fftfreq, fftshift

In [5]:
# matplotlib parameters 
large = 40; med = 20; small = 20
params = {'axes.titlesize': med,
          'axes.titlepad' : med,
          'legend.fontsize': med,
          'axes.labelsize': med ,
          'axes.titlesize': med ,
          'xtick.labelsize': med ,
          'ytick.labelsize': med ,
          'figure.titlesize': med}
plt.rcParams["font.family"] = "Helvetica"
plt.rcParams["font.serif"] = ["Helvetica Neue"]          
#plt.rcParams['text.usetex'] = True # need LaTeX. Change it to False if LaTeX is not installed in the system
plt.rcParams.update(params)
PI = np.pi
H_BAR = 6.626*10**(-34)/(2*PI)

folder_name_dict = { 
                    'run_0': 0.0,
                    'run_1': 0.1,
                    'run_2': 0.2,
                    'run_3': 0.3,
                    'run_4': 0.4,
                    'run_5': 0.5,
                    'run_6': 0.6,
                    'run_7': 0.7,
                    'run_8': 0.8,
                    'run_9': 0.9,
                    'run_10': 1.0}   

folder_name_lst = [keys for keys in folder_name_dict.keys()]
folder_name = folder_name_lst[0]
print("Folder name: ", folder_name)
total_number_of_b_folders = 128
b_index = total_number_of_b_folders - 1
path = '/Volumes/SASANKA_SSD/atomtronics_backup/atomtronics_modular_data/zoomed_2/' + str(folder_name) + '/b' + str(b_index)
os.chdir(path)
number_of_atoms = 10 * 1.e3
print("Number of atoms: ", number_of_atoms)
a_s_factor = np.load("a_s_factor.npy") 
print("a_s = ", a_s_factor)
V_SG = 30 # kHz
V_GD = 32 # kHz
print("V_SG = ", V_SG, "kHz")
print("V_GD = ", V_GD, "kHz")

Folder name:  run_0
Number of atoms:  10000.0
a_s =  0.0
V_SG =  30 kHz
V_GD =  32 kHz


In [6]:
b_start = 0
b_end = total_number_of_b_folders - 1
b_index = b_start
path = '/Volumes/SASANKA_SSD/atomtronics_backup/atomtronics_modular_data/zoomed_2/' + str(folder_name) + '/b' + str(b_index)
os.chdir(path)
V_SS_start_factor = np.load("source_bias.npy")
print("V_SS_start_factor = ", V_SS_start_factor)
b_index = b_end
path = '/Volumes/SASANKA_SSD/atomtronics_backup/atomtronics_modular_data/zoomed_2/' + str(folder_name) + '/b' + str(b_index)
os.chdir(path)
V_SS_end_factor = np.load("source_bias.npy")
print("V_SS_end_factor = ", V_SS_end_factor)
V_SS_lst_to_plot = np.linspace(V_SS_start_factor, V_SS_end_factor, total_number_of_b_folders)

V_SS_start_factor =  28.0
V_SS_end_factor =  30.5


In [8]:
PI = np.pi
H_BAR = 6.626*10**(-34)/(2*PI)
ATOM_MASS = 1.4192261*10**(-25) # kg
barrier_height_SG = V_SG # kHz
barrier_height_GD = V_GD # kHz
os.chdir(path)
gate_well_start = 0.0
gate_well_end = 4.8
print("Gate well width = ", gate_well_end - gate_well_start,r"$\mu m$")

single_particle_omega = np.sqrt(8*barrier_height_SG*10**3*H_BAR*2*PI/(ATOM_MASS*(gate_well_end*1.e-6 - gate_well_start*1.e-6)**2))
print(r"Single particle energy level frequency = ", single_particle_omega, "(rad/s)")
n_levels = int((barrier_height_SG*10**3*H_BAR*2*PI)/(H_BAR*7415) - 1/2)
print("Number of single particle energy levels in the gate well = ", n_levels)

single_particle_omega = 2*np.pi*1178
print("Time period = ", 2*np.pi/single_particle_omega * 1.e3, "s")

Gate well width =  4.8 $\mu m$
Single particle energy level frequency =  6973.716763152236 (rad/s)
Number of single particle energy levels in the gate well =  24
Time period =  0.8488964346349746 s


In [9]:
import os
import numpy as np
from scipy.fft import fft, fftfreq, fftshift
from scipy.ndimage import gaussian_filter1d, median_filter
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit

# Parameters
time_step = 1e-6  # seconds
tinit = 0.0e-3
tmax = 300e-3  # seconds

def gaussian(x, amplitude, center, width):
    """Gaussian function for curve fitting"""
    return amplitude * np.exp(-(x - center)**2 / (2 * width**2))

# Full time array from 0 to tmax
full_time_lst = np.arange(0.0, tmax, time_step)
start_idx = int(tinit / time_step)
end_idx = int(tmax / time_step)
print(f"Cropping from index {start_idx} to {end_idx} (time {tinit}s to {tmax}s)")

# Frequency range around target omega
omega_lower_limit = single_particle_omega - single_particle_omega / 10
omega_upper_limit = single_particle_omega + single_particle_omega / 10
b_index_lst = list(range(total_number_of_b_folders))

# smoothing_type = 'both'  # Options: 'gaussian', 'savgol', 'median', 'none', 'both'
# smoothing_type = 'gaussian'  # Options: 'gaussian', 'savgol', 'median', 'none', 'both'
smoothing_type = 'savgol'

for folder_name in folder_name_lst:
    print("Folder name: ", folder_name)
    peak_heights_drain = []  # From Gaussian fit
    raw_peak_heights_drain = []  # Direct peak heights
    peak_areas_drain = []
    b_with_valid_peaks = []

    for b_index in b_index_lst:
        print("b_index: ", b_index)
        path = f'/Volumes/SASANKA_SSD/atomtronics_backup/atomtronics_modular_data/zoomed_2/{folder_name}/b{b_index}'
        os.chdir(path)

        wavefunction = np.load("wavefunction_at_40um_vs_time_bias_idx_" + str(b_index) + ".npy")[:,1]
        density_drain = np.abs(wavefunction) ** 2

        density_drain = density_drain - np.mean(density_drain)

        density_drain = density_drain[start_idx:end_idx]
        print("Density shape after cropping:", density_drain.shape)

        # Apply FFT and smoothing as before
        N = len(density_drain)
        fft_values = fft(density_drain)
        frequencies = fftfreq(N, d=time_step)
        power_spectrum = np.abs(fft_values) ** 2
        frequencies_shifted = fftshift(frequencies)
        power_spectrum_shifted = fftshift(power_spectrum)

        # Apply smoothing (keeping existing smoothing code)
        if smoothing_type == 'savgol':
            window = 21 if N > 21 else N | 1
            power_spectrum_smooth = savgol_filter(power_spectrum_shifted, 
                                                window_length = window, 
                                                polyorder = 6)

        # Scale power spectrum for better numerical handling
        power_spectrum_smooth = power_spectrum_smooth * 1e8
        scale_factor = 1e8

        freqs_rad = 2 * np.pi * frequencies_shifted
        
        # Find indices for the three points
        freq_mask = (freqs_rad > omega_lower_limit) & (freqs_rad < omega_upper_limit)
        power_in_range = power_spectrum_smooth[freq_mask]
        freq_in_range = freqs_rad[freq_mask]
        
        if len(power_in_range) > 0:
            peak_idx = np.argmax(power_in_range)
            peak_freq = freq_in_range[peak_idx]
            raw_peak_height = power_in_range[peak_idx] / scale_factor  # Store raw peak height
            raw_peak_heights_drain.append(raw_peak_height)
            
            # Calculate symmetric points around peak
            left_freq = peak_freq - single_particle_omega / 10
            right_freq = peak_freq + single_particle_omega / 10

            # Find indices closest to these frequencies
            left_idx = np.argmin(np.abs(freqs_rad - left_freq))
            right_idx = np.argmin(np.abs(freqs_rad - right_freq))

            # Get the exact frequencies and power values at these three points
            freq_range = np.array([freqs_rad[left_idx], peak_freq, freqs_rad[right_idx]])
            power_range = np.array([power_spectrum_smooth[left_idx],
                                  power_in_range[peak_idx], 
                                  power_spectrum_smooth[right_idx]])

            try:
                # Calculate peak significance
                peak_height = power_range[1]  # Middle point (peak)
                background = min(power_range[0], power_range[2])  # Min of symmetric points
                peak_significance = peak_height / background if background > 0 else 0
                
                # Initial parameter guesses
                p0 = [power_range[1], peak_freq, single_particle_omega/10]
                bounds = ([0, omega_lower_limit, 0],
                          [np.inf, omega_upper_limit, single_particle_omega/5])

                if power_range[1] > max(power_range[0], power_range[2]) and peak_significance > 1.5:
                    popt, pcov = curve_fit(gaussian, freq_range, power_range, p0=p0, bounds=bounds)
                    peak_height = popt[0] / scale_factor  # Scale back to original units
                    peak_area = peak_height * popt[2] * np.sqrt(2 * np.pi)
                    b_with_valid_peaks.append(b_index)
                else:
                    peak_height = 0.0
                    peak_area = 0.0
                    
            except RuntimeError as e:
                print(f"Fit failed at b_index {b_index}: {str(e)}")
                peak_height = 0.0
                peak_area = 0.0
        else:
            raw_peak_heights_drain.append(0.0)
            peak_height = 0.0
            peak_area = 0.0

        peak_heights_drain.append(peak_height)
        peak_areas_drain.append(peak_area)

    save_path = '/Volumes/SASANKA_SSD/atomtronics_backup/atomtronics_modular_data/Figures/zoomed_2/'
    os.makedirs(save_path, exist_ok=True)
    os.chdir(save_path)
    
    # Save both raw and fitted peak heights
    np.save(f"{folder_name}_peak_FFT_heights_drain_signal_{int(omega_lower_limit)}_{int(omega_upper_limit)}.npy",
            np.array(peak_heights_drain))
    np.save(f"{folder_name}_raw_peak_FFT_heights_drain_signal_{int(omega_lower_limit)}_{int(omega_upper_limit)}.npy",
            np.array(raw_peak_heights_drain))
    np.save(f"{folder_name}_peak_FFT_areas_of_FFT_drain_signal_{int(omega_lower_limit)}_{int(omega_upper_limit)}.npy",
            np.array(peak_areas_drain))
    np.save(f"{folder_name}_b_with_peak_FFT_signal_{int(omega_lower_limit)}_{int(omega_upper_limit)}.npy",
            np.array(b_with_valid_peaks))

Cropping from index 0 to 300000 (time 0.0s to 0.3s)
Folder name:  run_0
b_index:  0
Density shape after cropping: (299999,)
b_index:  1
Density shape after cropping: (299999,)
b_index:  2
Density shape after cropping: (299999,)
b_index:  3


/var/folders/g5/w2_4q9x504zd1hn9cb6yqbm40000gn/T/ipykernel_91595/4046199715.py:110: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, pcov = curve_fit(gaussian, freq_range, power_range, p0=p0, bounds=bounds)


Density shape after cropping: (299999,)
b_index:  4
Density shape after cropping: (299999,)
b_index:  5
Density shape after cropping: (299999,)
b_index:  6
Density shape after cropping: (299999,)
b_index:  7
Density shape after cropping: (299999,)
b_index:  8
Density shape after cropping: (299999,)
b_index:  9
Density shape after cropping: (299999,)
b_index:  10
Density shape after cropping: (299999,)
b_index:  11
Density shape after cropping: (299999,)
b_index:  12
Density shape after cropping: (299999,)
b_index:  13
Density shape after cropping: (299999,)
b_index:  14
Density shape after cropping: (299999,)
b_index:  15
Density shape after cropping: (299999,)
b_index:  16
Density shape after cropping: (299999,)
b_index:  17
Density shape after cropping: (299999,)
b_index:  18
Density shape after cropping: (299999,)
b_index:  19
Density shape after cropping: (299999,)
b_index:  20
Density shape after cropping: (299999,)
b_index:  21
Density shape after cropping: (299999,)
b_index:  22